In [1]:
import os, math
import pandas as pd
import numpy as np

In [4]:
from gensim.models import Word2Vec as w2v
from os import path as osp

root = osp.dirname(os.getcwd())
model = w2v.load(osp.join(root,'ResumeMatchingMVP/model/stackexchange_model'))

In [5]:
def get_closest(word, n):
    '''Get n most similar words by words.'''
    #This function can easily be expanded to get similar words to phrases--
    #using sent2vec() method defined in WithWord2Vec notebook. 
    word = word.lower()
    words = [word]
    similar_vals = [1]
    try:
        similar_list = model.most_similar(positive=[word],topn=n)
        
        for tupl in similar_list:
            words.append(tupl[0])
            similar_vals.append(tupl[1])
    except:
        #If word not in vocabulary return same word and 1 similarity-- 
        #see initialisation of words, similarities.
        pass
    
    return words, similar_vals

In [9]:
cvs = pd.read_csv('D:\pre_course\Pratice\ResumeMatchingMVP\data\prc_data.csv', sep='\t')
cvs = cvs.set_index('Unnamed: 0')

prc_description = "software engineer c c java uml xamp agile defencecommunication client successful expand company develop air defence system information system target system communication system require additional software engineer support design evolution requirement software code identification test requirement development component test harness support design code review activity derisk design evolution qualification software engineer degree software engineering computer science math physics equivalent good degree pref 1st 1 meng equiv essential requirement software engineer experience follow object orient analysis design preferably uml unix linux window c c java html php candidate defence background experience agile development advantage experience xamp technology useful desirable requirement software engineer experience follow realtime design mathematical algorithm cod network protocol oo case tool structure design method datum voice network computer telephony integration cti appreciation follow test technique strategy configuration management personal attribute flexibility adaptability team player good communication skill discipline company provide excellent career prospect career development recent software graduate consider role keyword software engineer c c java realtime embed uml linux unix oo agile xamp defence communication system "

In [10]:
word_value = {}
similar_words_needed = 2
for word in prc_description.split():
    similar_words, similarity = get_closest(word, similar_words_needed)
    for i in range(len(similar_words)):
        word_value[similar_words[i]] = word_value.get(similar_words[i], 0)+similarity[i]
        print(similar_words[i], word_value[similar_words[i]])

software 1
engineer 1
c 1
c 2
java 1
uml 1
xamp 1
agile 1
defencecommunication 1
client 1
successful 1
expand 1
company 1
develop 1
air 1
defence 1
system 1
information 1
system 2
target 1
system 3
communication 1
system 4
require 1
additional 1
software 2
engineer 2
support 1
design 1
evolution 1
requirement 1
software 3
code 1
identification 1
test 1
requirement 2
development 1
component 1
test 2
harness 1
support 2
design 2
code 2
review 1
activity 1
derisk 1
design 3
evolution 2
qualification 1
software 4
engineer 3
degree 1
software 5
engineering 1
computer 1
science 1
math 1
physics 1
equivalent 1
good 1
degree 2
pref 1
1st 1
1 1
meng 1
equiv 1
essential 1
requirement 3
software 6
engineer 4
experience 1
follow 1
object 1
orient 1
analysis 1
design 4
preferably 1
uml 2
unix 1
linux 1
window 1
c 3
c 4
java 2
html 1
php 1
candidate 1
defence 2
background 1
experience 2
agile 2
development 2
advantage 1
experience 3
xamp 2
technology 1
useful 1
desirable 1
requirement 4
software 7
e

In [12]:
no_of_cv = 150

count = {}
idf = {}
for word in word_value.keys():
    count[word] = 0
    for i in range(no_of_cv):
        try:
            skills = cvs.iloc[i]['skill'] if isinstance(cvs.iloc[i]['skill'], str) else ""
            exp = cvs.iloc[i]['exp'] if isinstance(cvs.iloc[i]['exp'], str) else ""
            if word in skills.split() or word in exp.split():
                count[word] += 1
        except:
            pass
    if count[word] == 0:
        count[word] = 1
    idf[word] = math.log(no_of_cv / count[word])

print(count)
print(idf)

{'software': 1, 'engineer': 1, 'c': 1, 'java': 1, 'uml': 1, 'xamp': 1, 'agile': 1, 'defencecommunication': 1, 'client': 1, 'successful': 1, 'expand': 1, 'company': 1, 'develop': 1, 'air': 1, 'defence': 1, 'system': 1, 'information': 1, 'target': 1, 'communication': 1, 'require': 1, 'additional': 1, 'support': 1, 'design': 1, 'evolution': 1, 'requirement': 1, 'code': 1, 'identification': 1, 'test': 1, 'development': 1, 'component': 1, 'harness': 1, 'review': 1, 'activity': 1, 'derisk': 1, 'qualification': 1, 'degree': 1, 'engineering': 1, 'computer': 1, 'science': 1, 'math': 1, 'physics': 1, 'equivalent': 1, 'good': 1, 'pref': 1, '1st': 1, '1': 1, 'meng': 1, 'equiv': 1, 'essential': 1, 'experience': 1, 'follow': 1, 'object': 1, 'orient': 1, 'analysis': 1, 'preferably': 1, 'unix': 1, 'linux': 1, 'window': 1, 'html': 1, 'php': 1, 'candidate': 1, 'background': 1, 'advantage': 1, 'technology': 1, 'useful': 1, 'desirable': 1, 'realtime': 1, 'mathematical': 1, 'algorithm': 1, 'cod': 1, 'netwo

In [37]:
score = {}
for i in range(no_of_cv):
    score[i] = 0
    try:
        for word in word_value.keys():
            tf = cvs.loc(0)['skill'][i].split().count(word) + cvs.loc(0)['exp'][i].split().count(word)
            score[i] += word_value[word]*tf*idf[word]
    except:
        pass


# Step 2: Print raw scores for inspection
print("\nRaw Scores:")
for i in range(no_of_cv):
    print(f"CV{i}: {score[i]}")

print(type(score))


C:\Users\ritam\AppData\Local\Temp\ipykernel_17144\1447826629.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tf = cvs.loc(0)['skill'][i].split().count(word) + cvs.loc(0)['exp'][i].split().count(word)



Raw Scores:
CV0: 0
CV1: 230.48922352842774
CV2: 85.18079999963633
CV3: 120.25524705831015
CV4: 270.5743058811978
CV5: 1237.6269176417748
CV6: 145.30842352879142
CV7: 420.89336470408546
CV8: 340.7231999985452
CV9: 50.10635294096255
CV10: 1022.1695999956357
CV11: 95.20207058782886
CV12: 3757.976470572192
CV13: 240.51049411662027
CV14: 450.95717646866285
CV15: 470.99971764504795
CV16: 1022.1695999956356
CV17: 1032.1908705838287
CV18: 110.23397647011763
CV19: 95.20207058782887
CV20: 571.2124235269731
CV21: 85.18079999963635
CV22: 80.17016470554009
CV23: 726.542117643957
CV24: 0
CV25: 245.52112941071655
CV26: 115.24461176421389
CV27: 3757.976470572192
CV28: 1463.105505876107
CV29: 1022.169599995636
CV30: 45.0957176468663
CV31: 340.72319999854517
CV32: 345.7338352926415
CV33: 400.8508235277003
CV34: 110.23397647011764
CV35: 736.5633882321492
CV36: 636.3506823502244
CV37: 460.9784470568554
CV38: 295.627482351679
CV39: 55.116988235058805
CV40: 140.29778823469516
CV41: 3757.976470572192
CV42: 

In [42]:
# Step 3: Normalize scores (0–100)
max_score = max(score.values())
min_score = min(score.values())
print(max_score)
print(min_score)
normalized_score={}
if max_score == min_score:
    print("⚠️ All CVs have the same score. Normalization is not possible.")
    normalized_score = {i: 100 for i in score}
else:
    normalized_score = {i: (score[i] - min_score) / (max_score - min_score) * 100 for i in score}
print(type(normalized_score))

# print("\nRaw Scores:")
# for i in range(no_of_cv):
#     print(f"CV{i}: {normalized_score[i]}")

# Step 4: Sort CVs by normalized score (descending)
sorted_list = sorted(
    [(normalized_score[i], i) for i in range(no_of_cv)],
    reverse=True
)

# Step 5: Print sorted CVs with scores
print("\nSorted CVs by Normalized Score:")
for s, i in sorted_list:
    if list(cvs)[i] != '.DS_Store':  # You might want to adjust this condition
        print(f"CV{i}: {s:.2f}")


3757.976470572192
0
<class 'dict'>

Sorted CVs by Normalized Score:
CV41: 100.00
CV27: 100.00
CV12: 100.00
CV107: 92.00
CV130: 86.13
CV118: 84.80
CV121: 77.07
CV132: 66.00
CV51: 63.87
CV119: 58.53
CV84: 51.33
CV103: 45.33
CV111: 45.33
CV28: 38.93
CV49: 36.80
CV92: 36.00
CV5: 32.93
CV42: 28.13
CV17: 27.47
CV29: 27.20
CV10: 27.20
CV16: 27.20
CV45: 26.27
CV135: 22.13
CV128: 20.40
CV61: 19.60
CV35: 19.60
CV23: 19.33
CV98: 18.53
CV101: 17.87
CV62: 17.73
CV140: 17.20
CV36: 16.93
CV47: 16.13
CV20: 15.20
CV115: 14.53
CV109: 12.93
CV15: 12.53
CV37: 12.27
CV48: 12.13
CV14: 12.00
CV7: 11.20
CV88: 10.93
CV56: 10.80
CV33: 10.67
CV80: 10.13
CV43: 9.73
CV126: 9.20
CV32: 9.20
CV31: 9.07
CV8: 9.07
CV112: 8.93
CV143: 8.40
CV58: 8.27
CV147: 8.27
CV104: 8.27
CV57: 7.87
CV38: 7.87
CV60: 7.73
CV120: 7.60
CV79: 7.60
CV99: 7.47
CV78: 7.33
CV113: 7.33
CV4: 7.20
CV106: 7.20
CV102: 7.07
CV142: 6.93
CV55: 6.93
CV95: 6.80
CV74: 6.80
CV44: 6.80
CV69: 6.80
CV137: 6.67
CV25: 6.53
CV13: 6.40
CV63: 6.27
CV136: 6.13
CV1

In [20]:
sorted_list = []
for i in range(no_of_cv):
    sorted_list.append((score[i], i))
    
sorted_list.sort(reverse = True)

for s, i in sorted_list:
    if list(cvs)[i] != '.DS_Store':
        print(list(cvs)[i], ':', s)

cv135 : 3757.976470572192
cv122 : 3757.976470572192
cv109 : 3757.976470572192
cv195 : 3457.338352926416
cv215 : 3236.870399986181
cv204 : 3186.7640470452193
cv207 : 2896.1471999876344
cv217 : 2480.2644705776465
cv144 : 2400.0943058721064
cv205 : 2199.6688941082557
cv174 : 1929.094588227058
cv191 : 1703.6159999927274
cv199 : 1703.6159999927272
cv123 : 1463.105505876107
cv142 : 1382.9353411705667
cv181 : 1352.8715294059898
cv102 : 1237.6269176417748
cv136 : 1057.24404705431
cv113 : 1032.1908705838287
cv124 : 1022.169599995636
cv107 : 1022.1695999956357
cv112 : 1022.1695999956356
cv139 : 987.0951529369619
cv22 : 831.7654588199783
cv213 : 766.6271999967269
cv153 : 736.5633882321495
cv13 : 736.5633882321492
cv119 : 726.542117643957
cv187 : 696.4783058793795
cv19 : 671.4251294088983
cv154 : 666.4144941148016
cv224 : 646.3719529384167
cv130 : 636.3506823502244
cv140 : 606.2868705856466
cv116 : 571.2124235269731
cv201 : 546.1592470564917
cv197 : 486.03162352733676
cv111 : 470.99971764504795
cv